 Data Cleaning Script
In this step, we clean and prepare the supply chain dataset.
 We remove duplicates, merge date columns into a single date field,
and create a new column (Region_Group) to classify countries into regions
 such as Middle East, Europe, Latin America, and North America.
These steps help make the data consistent and ready for analysis.

In [ ]:
# importing the library for cleaning the data

import pandas as pd
# import numpy as np

###Leading the data from my pc

In [1]:
# loading the data from my pc
# loading the data from my pc
from google.colab import files
uploaded = files.upload()



Saving orders_and_shipments.csv to orders_and_shipments.csv


In [5]:
 # Veiwing the frist 10 row of the data
df = pd.read_csv('orders_and_shipments.csv')
df.head(10)

,Order ID,Order Item ID,Order YearMonth,Order Year,Order Month,Order Day,Order Time,Order Quantity,Product Department,Product Category,...,Customer Country,Warehouse Country,Shipment Year,Shipment Month,Shipment Day,Shipment Mode,Shipment Days - Scheduled,Gross Sales,Discount %,Profit
0,3535,8793,201502,2015,2,21,14:07,1,Fan Shop,Fishing,...,Mexico,Puerto Rico,2015,2,27,Standard Class,4,400,0.25,200
1,4133,10320,201503,2015,3,2,7:37,1,Fan Shop,Fishing,...,Brazil,Puerto Rico,2015,3,6,Standard Class,4,400,0.09,200
2,7396,18517,201504,2015,4,18,22:47,1,Fan Shop,Fishing,...,Mexico,Puerto Rico,2015,4,20,Standard Class,4,400,0.06,200
3,11026,27608,201506,2015,6,10,22:32,1,Fan Shop,Fishing,...,Denmark,Puerto Rico,2015,6,12,Standard Class,4,400,0.15,200
4,11026,27609,201506,2015,6,10,22:32,1,Fan Shop,Fishing,...,Denmark,Puerto Rico,2015,6,12,Standard Class,4,400,0.13,200
5,11026,27607,201506,2015,6,10,22:32,1,Fan Shop,Fishing,...,Denmark,Puerto Rico,2015,6,12,Standard Class,4,400,0.16,200
6,19273,48192,201510,2015,10,9,7:50,1,Fan Shop,Fishing,...,Netherlands,Puerto Rico,2015,10,13,Standard Class,4,400,0.06,200
7,19566,48912,201510,2015,10,13,14:29,1,Fan Shop,Fishing,...,Germany,Puerto Rico,2015,10,15,Standard Class,4,400,0.12,200
8,21215,53077,201511,2015,11,6,16:12,1,Fan Shop,Fishing,...,China,Puerto Rico,2015,11,12,Standard Class,4,400,0.04,200
9,21846,54626,201511,2015,11,15,21:16,1,Fan Shop,Fishing,...,Indonesia,Puerto Rico,2015,11,17,Standard Class,4,400,0.02,200


In [7]:
# Count rows and  basic sanity
df.shape

# Check for missing values and  nulls
df.isnull().sum()


,0
Order ID,0
Order Item ID,0
Order YearMonth,0
Order Year,0
Order Month,0
Order Day,0
Order Time,0
Order Quantity,0
Product Department,0
Product Category,0


#🧹Columns Removed and Why

- Order ID — This is just a unique reference for each order and doesn’t help analyze delays, so it’s not needed.

- Order Item ID — Like Order ID, this is only for internal tracking and doesn’t provide insights into shipments or delays.

- Order YearMonth / Order YearMonth — These duplicate information already available from the order date, so keeping them would be redundant.

- Gross Sales — Sales figures are useful for business performance but don’t affect delivery timing, so we removed them to keep the dataset focused.

- Discount % — This relates to pricing and promotions, not delivery delays, so it’s not relevant for our analysis.

- Profit — Profit shows financial performance, but it doesn’t explain shipment delays, so we left it out to keep the focus on logistics.

In [15]:
# Strip all leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Now drop the unnecessary columns
df = df.drop([
    'Order ID',
    'Order Item ID',
    'Order YearMonth',
    'Gross Sales',
    'Discount %',
    'Profit'
], axis=1)





### Combine three colums
Combine the separate 'Order Year', 'Order Month', and 'Order Day' columns
 into a single 'Order Date' column of datetime type.
This makes it much easier to perform time-based analysis, filtering, and visualization'


In [ ]:
# Combine 'Order Year', 'Order Month', and 'Order Day' into a single 'Order Date' column.

df['Order Date'] = pd.to_datetime(
    df[['Order Year', 'Order Month', 'Order Day']]
    .astype(str)
    .agg('-'.join, axis=1)
)

# Drop the original columns now that we have 'Order Date'
df = df.drop(columns=['Order Year', 'Order Month', 'Order Day'])

In [23]:
df.head()
df.columns


Index(['Order ID ', ' Order Item ID ', ' Order YearMonth ', ' Order Year ',
       ' Order Month ', ' Order Day ', 'Order Time', 'Order Quantity',
       'Product Department', 'Product Category', 'Product Name',
       ' Customer ID ', 'Customer Market', 'Customer Region',
       'Customer Country', 'Warehouse Country', 'Shipment Year',
       'Shipment Month', 'Shipment Day', 'Shipment Mode',
       ' Shipment Days - Scheduled ', ' Gross Sales ', ' Discount % ',
       ' Profit '],
      dtype='object')

###Merging Shipment Date Columns:
We combine the separate Shipment Year, Shipment Month, and Shipment Day columns into a single Shipment Date column of datetime type.
This simplifies the dataset and makes it easier to perform time-based analysis, such as calculating delivery durations, filtering by date ranges, and creating visualizations.


In [24]:
# Combine 'Shipment Year', 'Shipment Month', and 'Shipment Day' into a single 'Shipment Date' column.

df['Shipment Date'] = pd.to_datetime(
    df[['Shipment Year', 'Shipment Month', 'Shipment Day']]
    .astype(str)
    .agg('-'.join, axis=1)
)

# Drop the original columns now that we have 'Shipment Date'
df = df.drop(columns=['Shipment Year', 'Shipment Month', 'Shipment Day'])


### We are creating the Region column:
We are creating a Region column to group countries into broader regions like North America, Latin America, Europe, Middle East, Asia, Africa, and Oceania.
This makes the data easier to analyze and visualize, helps us compare trends across regions, and keeps the dataset organized. Any countries not included in the mapping are labeled as 'Other' so we don’t lose any information.

In [ ]:
# Dictionary mapping countries to regions
region_mapping = {
    # North America
    'USA': 'North America',
    'Canada': 'North America',

    # Latin America
    'Mexico': 'Latin America',
    'Brazil': 'Latin America',
    'Guatemala': 'Latin America',
    'Panama': 'Latin America',
    'Chile': 'Latin America',
    'Cuba': 'Latin America',
    'Nicaragua': 'Latin America',
    'Honduras': 'Latin America',
    'Dominican Republic': 'Latin America',
    'Venezuela': 'Latin America',
    'Argentina': 'Latin America',
    'Colombia': 'Latin America',
    'Peru': 'Latin America',
    'Ecuador': 'Latin America',
    'Bolivia': 'Latin America',
    'Paraguay': 'Latin America',
    'Uruguay': 'Latin America',
    'El Salvador': 'Latin America',
    'Haiti': 'Latin America',
    'Jamaica': 'Latin America',
    'Trinidad and Tobago': 'Latin America',
    'Guyana': 'Latin America',
    'Barbados': 'Latin America',
    'Guadalupe': 'Latin America',
    'Martinique': 'Latin America',
    'Belize': 'Latin America',
    'Costa Rica': 'Latin America',
    'French Guiana': 'Latin America',

    # Europe
    'Denmark': 'Europe',
    'Netherlands': 'Europe',
    'Germany': 'Europe',
    'Hungary': 'Europe',
    'Poland': 'Europe',
    'France': 'Europe',
    'Sweden': 'Europe',
    'United Kingdom': 'Europe',
    'Italy': 'Europe',
    'Spain': 'Europe',
    'Belgium': 'Europe',
    'Switzerland': 'Europe',
    'Austria': 'Europe',
    'Norway': 'Europe',
    'Finland': 'Europe',
    'Ireland': 'Europe',
    'Portugal': 'Europe',
    'Czech Republic': 'Europe',
    'Greece': 'Europe',
    'Croatia': 'Europe',
    'Romania': 'Europe',
    'Belarus': 'Europe',
    'Albania': 'Europe',
    'Georgia': 'Europe',
    'Ukraine': 'Europe',
    'Bulgaria': 'Europe',
    'Slovakia': 'Europe',
    'Estonia': 'Europe',
    'Lithuania': 'Europe',
    'Montenegro': 'Europe',
    'Macedonia': 'Europe',
    'Czech Republic (Czechia)': 'Europe',
    'Bosnia and Herzegovina': 'Europe',
    'Moldova': 'Europe',

    # Middle East
    'Iran': 'Middle East',
    'Iraq': 'Middle East',
    'Turkey': 'Middle East',
    'Afghanistan': 'Middle East',
    'Saudi Arabia': 'Middle East',
    'UAE': 'Middle East',
    'Jordan': 'Middle East',
    'Lebanon': 'Middle East',
    'Syria': 'Middle East',
    'Palestine': 'Middle East',
    'Egypt': 'Middle East',
    'Qatar': 'Middle East',
    'Kuwait': 'Middle East',
    'Oman': 'Middle East',
    'Bahrain': 'Middle East',
    'Yemen': 'Middle East',
    'Israel': 'Middle East',
    'Liban': 'Middle East',

    # Asia (excluding Middle East)
    'China': 'Asia',
    'India': 'Asia',
    'Indonesia': 'Asia',
    'Pakistan': 'Asia',
    'South Korea': 'Asia',
    'Singapore': 'Asia',
    'Japan': 'Asia',
    'Thailand': 'Asia',
    'Malaysia': 'Asia',
    'Vietnam': 'Asia',
    'Philippines': 'Asia',
    'Bangladesh': 'Asia',
    'Nepal': 'Asia',
    'Sri Lanka': 'Asia',
    'Myanmar': 'Asia',
    'Cambodia': 'Asia',
    'Laos': 'Asia',
    'Kazakhstan': 'Asia',
    'Uzbekistan': 'Asia',
    'Kyrgyzstan': 'Asia',
    'Taiwan': 'Asia',
    'Mongolia': 'Asia',

    # Africa
    'Sudan': 'Africa',
    'Democratic Republic of Congo': 'Africa',
    'Togo': 'Africa',
    'Madagascar': 'Africa',
    'Morocco': 'Africa',
    'Niger': 'Africa',
    'South Africa': 'Africa',
    "Cote d'Ivoire": 'Africa',
    'Nigeria': 'Africa',
    'Kenya': 'Africa',
    'Ghana': 'Africa',
    'Ethiopia': 'Africa',
    'Cameroon': 'Africa',
    'Uganda': 'Africa',
    'Algeria': 'Africa',
    'Tanzania': 'Africa',
    'Senegal': 'Africa',
    'Gabon': 'Africa',
    'Angola': 'Africa',
    'Mali': 'Africa',
    'Guinea': 'Africa',
    'Rwanda': 'Africa',
    'Libya': 'Africa',
    'Mozambique': 'Africa',
    'Lesotho': 'Africa',
    'Zambia': 'Africa',
    'Mauritania': 'Africa',
    'Sierra Leona': 'Africa',
    'Namibia': 'Africa',
    'Republic of Congo': 'Africa',

    # Oceania
    'Australia': 'Oceania',
    'New Zealand': 'Oceania',
    'Fiji': 'Oceania',
    'Papua New Guinea': 'Oceania'
}

# Create a 'Region' column based on the country
df['Region'] = df['Customer Country'].map(region_mapping)

# Fill any unmapped countries with 'Other'
df['Region'] = df['Region'].fillna('Other')

# Check the result
df[['Customer Country', 'Region']].head(20)


,Customer Country,Region
0,Mexico,Latin America
1,Brazil,Latin America
2,Mexico,Latin America
3,Denmark,Europe
4,Denmark,Europe
5,Denmark,Europe
6,Netherlands,Europe
7,Germany,Europe
8,China,Asia
9,Indonesia,Asia


In [53]:
# Show all rows where Region is 'Other'
df_other = df[df['Region'] == 'Other']

# Display unique countries that were not mapped
df_other['Customer Country'].unique()



array(['Dominican�Republic', 'Cote d�Ivoire', 'Russia', 'Per�',
       'Hong Kong', 'Somalia', 'Benin', 'Tunisia', 'Zimbabwe', 'Liberia',
       'Guinea-Bissau', 'Azerbaijan', 'Gambia'], dtype=object)

We cleaned the Customer Country column to fix special characters, extra spaces, and known typos.
This ensures all country names are consistent and can be correctly grouped into regions later.

In [54]:
# Fix special characters and typos
df['Customer Country'] = df['Customer Country'].str.replace('ï¿½', "'", regex=False)
df['Customer Country'] = df['Customer Country'].str.replace('Per�', 'Peru', regex=False)
df['Customer Country'] = df['Customer Country'].str.replace('Ben�n', 'Benin', regex=False)
df['Customer Country'] = df['Customer Country'].str.strip()

# Add missing countries to the region mapping
region_mapping.update({
    'Dominican Republic': 'Latin America',
    'Cote d\'Ivoire': 'Africa',
    'Russia': 'Europe',
    'Hong Kong': 'Asia',
    'Somalia': 'Africa',
    'Tunisia': 'Africa',
    'Zimbabwe': 'Africa',
    'Liberia': 'Africa',
    'Guinea-Bissau': 'Africa',
    'Azerbaijan': 'Asia',
    'Gambia': 'Africa'
})

# Apply region mapping
df['Region'] = df['Customer Country'].map(region_mapping)
df['Region'] = df['Region'].fillna('Other')

# Check unmapped countries
df[df['Region'] == 'Other']['Customer Country'].unique()


array(['Dominican�Republic', 'Cote d�Ivoire', 'Benin'], dtype=object)

In this step, we are fixing the remaining country names that have special characters or encoding issues, so they match correctly with our region mapping. After that, we reassign each country to its region and check if there are any countries that still don’t belong to a region.

In [61]:
# Fix the remaining special characters in country names
df['Customer Country'] = df['Customer Country'].str.replace('Dominican�Republic', 'Dominican Republic', regex=False)
df['Customer Country'] = df['Customer Country'].str.replace('Cote d�Ivoire', "Cote d'Ivoire", regex=False)

# Apply the region mapping again
df['Region'] = df['Customer Country'].map(region_mapping)
df['Region'] = df['Region'].fillna('Other')

# Check if any unmapped countries remain
df[df['Region'] == 'Other']['Customer Country'].unique()


array(['Benin'], dtype=object)

In [62]:
# saving the data after all the cleaning
df.to_csv('corders_and_shipments_cleaned.csv', index=False)